In [1]:
import collections
import numpy as np
import time
import pprint
import pandas as pd
from sklearn import linear_model
import scipy.stats
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import modules
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os, sys, getopt
import xml.etree.ElementTree as ET

import pandas as pd
from pandas import Series, DataFrame

import tarfile

In [3]:
# verify spark binary in HDFS
!hadoop fs -ls /FIXME/twitterParquet

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Found 4 items
drwxr-xr-x   - ytesfaye ytesfaye          0 2015-05-01 17:11 /user/ytesfaye/twitterParquet/april.parquet
drwxr-xr-x   - ytesfaye ytesfaye          0 2015-05-01 00:31 /user/ytesfaye/twitterParquet/march.parquet
drwxr-xr-x   - ytesfaye ytesfaye          0 2015-06-02 16:45 /user/ytesfaye/twitterParquet/may.parquet
drwxr-xr-x   - ytesfaye ytesfaye          0 2015-06-02 16:51 /user/ytesfaye/twitterParquet/month=4.parquet


In [4]:
# load data and register a table
tweets = sqlCtx.parquetFile('hdfs://cdh:8020/FIXME/twitterParquet/may.parquet/part-r-00001.parquet')
tweets.registerTempTable('tweets')
tweets.count()
#tweets = sqlCtx.parquetFile('hdfs://cdh:8020/FIXME/twitterParquet/may.parquet')

/usr/local/spark-1.4.1-bin-hadoop2.4/python/pyspark/sql/context.py:384: UserWarning: parquetFile is deprecated. Use read.parquet() instead.
  warnings.warn("parquetFile is deprecated. Use read.parquet() instead.")


In [7]:
# ensure utf8 encoding
import sys    
reload(sys)  
sys.setdefaultencoding('utf8')

In [8]:
# select arabic tweets that are non-retweets 
arabic_tweets = sqlCtx.sql("select id_str, text from tweets where lang = 'ar' AND retweet_count = '0' AND text NOT LIKE 'RT @%'")

In [9]:
# display the number of filtered results
arabic_tweets.count()

3919

In [21]:
# mimic `mkdir --parents`
def mkdir_p(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
# mimic `chmod -R`
def chmod_recursive(directory, perms):
    for root, dirs, files in os.walk(directory):  
        
        # process all files and directories at once
        dirs.extend(files)
        
        # traverse and update
        for pathname in dirs:  
            os.chmod(os.path.join(root, pathname), perms)


In [13]:
# output
dir_root = '/ipython/arabic'
dir_files = 'may_part-r-00001'

In [15]:
# ensure paths exist
mkdir_p(dir_root)
mkdir_p(os.path.join(dir_root, dir_files))
chmod_recursive(dir_root, 0777)

In [16]:
# build filename to include date
filename = '{}/{}_{}.csv'.format(dir_root, dir_files, time.strftime("%Y.%m.%d"))

In [17]:
# convert spark->pandas dataframe to save locally
df = arabic_tweets.toPandas()

In [18]:
# save 'text' column in each row of dataframe as dir_root/dir_files/<tweet-id>.txt
def save_tweet_as_text(row, field="text", minlen=45):
    
    # format the filename
    filename = '{}/{}/{}.text'.format(dir_root, dir_files, row['id_str'])
    
    # only save longer tweets
    if (len(row[field]) >= minlen):
        with open(filename, "w") as text_file:
            text_file.write("{}".format(row[field]))
        
    # return the filename to the calling method
    return filename


# compress source directory into an output filename.tar.gz
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [19]:
# save each tweet as an individual text file
df.apply(lambda row: save_tweet_as_text(row), axis=1)

In [20]:
# create an archive of all tweets
make_tarfile('{}/{}.tar.gz'.format(dir_root, dir_files), '{}/{}'.format(dir_root, dir_files))